# Obtaining RDMs From fMRI Data

This notebook contains code to obtain representational dissimilarity matrices from fMRI responses in both experimental conditions (picture condition and sentence condition).

In [ ]:
# importing necessary libraries
import os
import numpy as np
from scipy.io import loadmat
from scipy.spatial.distance import pdist, squareform
from src.utils import open_json, dict_to_json
from src.paths import ROOT

In [ ]:
# function to extract netowrk-wise representations

def get_network_activations(fmri_data, network_name):

  """
    This function takes as input the loaded matlab file with fMRI responses from one participant (fmri_data)
    and outputs an n_concepts x n_ROIs matrix with the fMRI responses from a specific brain network
    (network name).
  """

  networks = [atlas[0] for atlas in fmri_data['meta']['atlases'][0][0][0]]
  nw_index = networks.index(network_name)

  nw_columns = fmri_data['meta']['roiColumns'][0][0][0][nw_index]
  column_indexes = np.concatenate([nw_columns[roi][0].flatten()-1 for roi in range(len(nw_columns))], axis=0)

  network_responses = fmri_data['examples'][:, column_indexes]

  return network_responses

In [ ]:
# obtaining a list of participant IDs based on folder names
participants = os.listdir(ROOT / 'data/Pereira/subjects')

In [ ]:
# creating a dictionary to store RDMs

responses = {'picture': {'visual': [],
                        'languageLH': [],
                        'languageRH': []},
            'sentence': {'visual': [],
                        'languageLH': [],
                        'languageRH': []}}

## Picture condition

We first populate the dictionary with RDMs from the fMRI data collected in the picture condition.

In [ ]:
# populating the dictionary with matrices

for participant in participants:
    part_data  = loadmat(ROOT / 'data/Pereira/subjects' / participant / 'data_180concepts_pictures.mat')
    for network in ['visual', 'languageLH', 'languageRH']:
        activations = get_network_activations(part_data, network)
        responses['picture'][network].append(activations)
    

In [ ]:
# turning the matrices into RDMs

for network in ['visual', 'languageLH', 'languageRH']:
    network_rdm = np.empty((16, 180, 180))
    for i in range(len(participants)):
        network_rdm[i] = squareform(pdist(responses['picture'][network][i], metric='cosine'))
    avg_rdm = network_rdm.mean(axis=0).tolist()
    responses['picture'][network] = avg_rdm

## Sentence condition

We then add RDMs from the fMRI data collected in the sentence condition.

In [21]:
for participant in participants:
    part_data  = loadmat(ROOT / 'data/Pereira/subjects' / participant / 'data_180concepts_sentences.mat')
    for network in ['visual', 'languageLH', 'languageRH']:
        activations = get_network_activations(part_data, network)
        responses['sentence'][network].append(activations)
    

In [22]:
for network in ['visual', 'languageLH', 'languageRH']:
    network_rdm = np.empty((16, 180, 180))
    for i in range(len(participants)):
        network_rdm[i] = squareform(pdist(responses['sentence'][network][i], metric='cosine'))
    avg_rdm = network_rdm.mean(axis=0).tolist()
    responses['sentence'][network] = avg_rdm

In [ ]:
# saving the dict storing RDMs

dict_to_json(responses, 'participant_rdms.json')